# CellphoneDB scoring protocol
**B-cell signaling example**

In [ ]:
%%capture
# pip install --force-reinstall "git+https://github.com/ventolab/CellphoneDB.git@scoring"

### Load scanpy object

In [ ]:
import scanpy as sc
import pandas as pd
adata = sc.read('/Users/rp23/Downloads/kevin_bcells_stroma/adata_subset_Bcells_stroma.h5ad')
adata.shape
norm_matrix = pd.DataFrame(adata.X.todense(),
                           columns = list(adata.var.index),
                           index = list(adata.obs.index)).transpose()
norm_matrix.shape
metadata = adata.obs

# Remove scanpy object to save memory
del adata

### Load CellphoneDB

In [ ]:
import os
# The default version of CellphoneDB data is the latest one, but you can change it to a previous version 
# at any point in this notebook (by re-setting the value of cpdb_version variable). 
# Please note that the format of the database from version v4.1.0 is incompatible with that of previous 
# versions, hence the lowest version number you may choose in this notebook is v4.1.0
cpdb_version = "v4.1.0"
# cpdb_dir will contain the *_input.csv and cellphonedb.zip files that you will download from https://github.com/ventolab/cellphonedb-data
# At the very least, please replace <your_user_id> with your user id
cpdb_dir = os.path.join("/Users/rp23/.cpdb/releases", cpdb_version)
cpdb_file_path = os.path.join(cpdb_dir, "cellphonedb.zip")

### Calculate interaction scores

In [ ]:
from cellphonedb.utils import scoring_utils
# Obtain interaction scores
interaction_scores_dict = scoring_utils.score_interactions_based_on_participant_expressions_product(
            cpdb_file_path, norm_matrix, 'gene_name', metadata, 0.1, 'cell.labels', 4)

### List all cell-pairs comparison
Results are stored as as dictionary of dataframes, each dataframe is named after the cells being analyzed for cell-cell communication. \
Beware you will find `cell_A|cell_B` but not `cell_B|cell_A`. Each dataframe contains the partners swapped to compare interactions in both directions.

In [ ]:
list(interaction_scores_dict.keys())[0:10]

### Example of how to query results
Ordering results by the score

In [ ]:
example_table = interaction_scores_dict['endosteal fibroblast|osteoclast'].sort_values('Score',
                                                                            ascending = False)

In [ ]:
example_table.head(50)

____